In [81]:
from collections import defaultdict
from collections import deque
import sys

Class Node -> to store all values that corresponds to the node/id.
All necessary variables are declared below:

In [82]:
class Node:
    def __init__(self,id,name,dur,es,ef,ls,lf) -> None:
        self.name = name
        self.id = id ; self.du = dur
        self.es = es; self.ef = ef
        self.ls = ls; self.lf = lf
    
nodes = {}
q1 = deque()
q2 = deque()
forwardPass = defaultdict(list)
backwardPass = defaultdict(list)
criticalPath = []
totalDuration = 0

Reading input file and Storing all the corresponding value.
nodes dictionary used to store the object of Node class.
All the id appended to the queue which have no predecessors.

In [83]:
path = './activity.txt'
for lines in open(path):
    line = lines.rstrip('\n').split(',')
    id = int(line[0]); name = line[1]; duration = int(line[2])
    ef = 0
    if len(line) == 4:
        predecessors = line[3].split(';')
        for x in predecessors:
            x = int(x)
            forwardPass[x].append(id)
            backwardPass[id].append(x)
    else:
        q1.append(id)
        ef = duration

    nodes[id] = Node(id,name,duration,0,ef,0,sys.maxsize)

ForwardPass:
    Traverse through the forwardPass using BFS.
    Update es and ef --- es is updated with maximum value of its predecessors.
    outDegree is used to find the leaves from which we'll start backwardPass.

In [84]:
while q1:
    u = q1.popleft()
    ef = nodes[u].ef
    for v in forwardPass[u]:
        if nodes[v].es < ef:
            nodes[v].es = ef
            nodes[v].ef = nodes[v].es + nodes[v].du
            q1.append(v)

    if len(forwardPass[u]) == 0:
        totalDuration = max(totalDuration,nodes[u].ef)
        q2.append(u)

BackwardPass:
    All leaf nodes are added to the queue and corresponding ls , lf is calculated.
    Using BFS forwardPass is traversed in backward direction.
    lf is calculated as minimum(ls) of its successors.

In [85]:
while q2:
    u = q2.popleft()

    if len(forwardPass[u]) == 0:
        nodes[u].lf = totalDuration
        nodes[u].ls = nodes[u].lf - nodes[u].du
        q1.append(u)

    ls = nodes[u].ls
    for v in backwardPass[u]:
        if nodes[v].lf > ls:
            nodes[v].lf = ls
            nodes[v].ls = nodes[v].lf - nodes[v].du
            q2.append(v)

In [86]:
while q1:
    u = q1.popleft()
    if nodes[u].es == nodes[u].ls and nodes[u].name not in criticalPath:
        criticalPath.append(nodes[u].name)
        for v in backwardPass[u]:
            if nodes[v].name not in criticalPath:
                q1.append(v)
            

In [87]:
def printNodes(nodes):
    for key,value in nodes.items():
        print('id = {},node = {},duration = {},ES = {},EF = {},LS = {},LF = {}'
            .format(key,value.name,value.du,value.es,value.ef,value.ls,value.lf))
            
printNodes(nodes)

id = 1,node = A,duration = 3,ES = 0,EF = 3,LS = 0,LF = 3
id = 2,node = B,duration = 4,ES = 3,EF = 7,LS = 3,LF = 7
id = 3,node = C,duration = 2,ES = 3,EF = 5,LS = 9,LF = 11
id = 4,node = D,duration = 5,ES = 7,EF = 12,LS = 7,LF = 12
id = 5,node = E,duration = 1,ES = 5,EF = 6,LS = 11,LF = 12
id = 6,node = F,duration = 2,ES = 5,EF = 7,LS = 14,LF = 16
id = 7,node = G,duration = 4,ES = 12,EF = 16,LS = 12,LF = 16
id = 8,node = H,duration = 3,ES = 16,EF = 19,LS = 16,LF = 19


In [88]:
for i , e  in reversed(list(enumerate(criticalPath))):
    print(e,end = '')
    if i != 0:
        print('->',end='')

A->B->D->G->H